In [1]:
import os
import shutil

root_train_dir = "/home/xiaoyzhu/notebooks/currency_detector/data/train"
root_test_dir = "/home/xiaoyzhu/notebooks/currency_detector/data/test"
root_validation_dir = "/home/xiaoyzhu/notebooks/currency_detector/data/validation"
root_visualizaion_dir = "/home/xiaoyzhu/notebooks/currency_detector/data/visualization"



In [2]:

from keras import applications
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.layers import Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

img_width, img_height = 224,224
train_data_dir = root_train_dir
validation_data_dir = root_validation_dir
nb_train_samples = 1672
nb_validation_samples = 560
train_steps = 100 # 1672 training samples/batch size of 32 = 52 steps. We are doing heavy data processing so put 500 here
validation_steps = 20 # 560 validation samples/batch size of 32 = 10 steps. We put 20 for validation steps
batch_size = 32
epochs = 100

def build_model():
    # constructing the model
    model = applications.Xception(weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3),
                                  pooling='avg')
    
    model = applications.mobilenet.MobileNet(weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3),
                                  pooling='avg')

    # only train the last 2 layers
    for layer in model.layers[:-10]:
        layer.trainable = False

    # Adding custom Layers
    x = model.output
    # x = Flatten()(x)
    predictions = Dense(14, activation="softmax")(x)

    # creating the final model
    model_final = Model(inputs=model.input, outputs=predictions)
    
    return model_final

model_final = build_model()
# compile the model
model_final.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=0.001), metrics=["accuracy"])

# Initiate the train and test generators with data Augumentation
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    fill_mode="nearest",
    zoom_range=0.3,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    rotation_range=360)

validation_datagen = ImageDataGenerator(
    rescale=1. / 255,
    fill_mode="nearest",
    zoom_range=0.3,
    rotation_range=30)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    # save_to_dir = root_visualizaion_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical")

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    class_mode="categorical")

# Save the model according to the conditions
checkpoint = ModelCheckpoint("currency_detector.h5", monitor='val_loss', verbose=1, save_best_only=True,
                             save_weights_only=False,
                             mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

tensorboard = TensorBoard(log_dir='/home/xiaoyzhu/notebooks/currency_detector/tensorboard/',  write_images = True)

# Train the model
model_final.fit_generator(
    train_generator,
    steps_per_epoch = train_steps,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps = validation_steps,
    workers=16,
    callbacks=[checkpoint, early, tensorboard])

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 2092 images belonging to 14 classes.
Found 420 images belonging to 14 classes.
Epoch 1/100
100/100 [==============================] - 24s 244ms/step - loss: 0.9058 - acc: 0.7195 - val_loss: 3.3048 - val_acc: 0.2827

Epoch 00001: val_loss improved from inf to 3.30481, saving model to currency_detector.h5
Epoch 2/100
100/100 [==============================] - 21s 209ms/step - loss: 0.3012 - acc: 0.8994 - val_loss: 1.8604 - val_acc: 0.5441

Epoch 00002: val_loss improved from 3.30481 to 1.86043, saving model to currency_detector.h5
Epoch 3/100
100/100 [==============================] - 22s 218ms/step - loss: 0.1950 - acc: 0.9364 - val_loss: 2.7682 - val_acc: 0.3682

Epoch 00003: val_loss did not improve
Epoch 4/100
100/100 [==============================] - 21s 211ms/step - loss: 0.1523 - acc: 0.9494 - val_loss: 5.2743 - val_acc: 0.3203

Epoch 00004: val_loss did not improve
Epoch 5/100
100/100 [==============================] - 22s 216ms/step - loss: 0.1818 - acc: 0.9415 - val_loss

In [3]:
import onnxmltools
import coremltools

# install from https://github.com/onnx/onnxmltools and https://github.com/apple/coremltools

model_coreml = coremltools.converters.keras.convert("currency_detector.h5", image_scale = 1./255)
model_onnx = onnxmltools.convert.convert_coreml(model_coreml, "currency_detector")

# Save as protobuf
onnxmltools.utils.save_model(model_onnx, "currency_detector" + ".onnx")

ValueError: Unknown activation function:relu6

# Utils to Move data around

In [ ]:
# utility to move data around
def move_files_subfolders(root_src_dir, root_target_dir, operation, image_number):
    for src_dir, dirs, files in os.walk(root_src_dir):
        num_temp = 0
        dst_dir = src_dir.replace(root_src_dir, root_target_dir)
        if not os.path.exists(dst_dir):
            os.mkdir(dst_dir)
        for individual_file in files:
            if num_temp < image_number:
                src_file = os.path.join(src_dir, individual_file)
                dst_file = os.path.join(dst_dir, individual_file)
                if os.path.exists(dst_file):
                    os.remove(dst_file)
                if operation is 'copy':
                    shutil.copy(src_file, dst_dir)
                elif operation is 'move':
                    shutil.move(src_file, dst_dir)
                num_temp += 1
            else:
                break

#move_files_subfolders(root_test_dir, root_train_dir, 'move', 20)
#move_files_subfolders(root_validation_dir, root_train_dir, 'move', 10)
